  # Neural Network Training for the MtM approach

In [12]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt 

plt.style.use('bmh')

import matplotlib
matplotlib.rcParams.update({'font.size': 16})

from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%aimport IS_misc_functions
from IS_misc_functions import *
from misc_functions import *
from actuarial_training import *
from MtM_training import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler,normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create the neural network

In [13]:
def build_model(x,nr_neurons = 32,regul = 0.005):

    model = keras.Sequential([
        layers.Dense(nr_neurons,kernel_regularizer=regularizers.l2(regul),activation='relu',input_shape=[600]),
        #layers.Dropout(0.3),
        layers.Dense(nr_neurons,kernel_regularizer=regularizers.l2(regul), activation='relu'),
        #layers.Dropout(0.3),
        layers.Dense(nr_neurons,kernel_regularizer=regularizers.l2(regul), activation='relu'),
        #layers.Dropout(0.3),
        layers.Dense(nr_neurons,kernel_regularizer=regularizers.l2(regul), activation='relu'),
        #layers.Dropout(0.3),
        layers.Dense(1)
      ])
    opt1 = tf.keras.optimizers.Adam(learning_rate = 0.00005, beta_1=0.9, beta_2=0.999)
    model.compile(loss='mse',
                    optimizer=opt1,
                    metrics=['mae', 'mse'])
    scaler = StandardScaler().fit(x)
    return model, scaler

### Load the Training Data

In [14]:
# # Read Data
def read_data(xpath,ypath):
    df_x = pd.read_csv(xpath, header=None)
    df_x = df_x.dropna()
    df_x = df_x.iloc[:,1:]

    df_y = pd.read_csv(ypath, header=None)
    df_y = df_y.dropna()
    df_y = df_y.iloc[:,1:]
    
    return df_x,df_y
x,y = read_data("csv/X_mtm_1.csv","csv/Y_mtm_1.csv")
for i in range(13,15):
    x_new,y_new = read_data("csv/X_mtm_"+str(i)+".csv","csv/Y_mtm_"+str(i)+".csv")
    x = pd.concat([x,x_new])
    y = pd.concat([y,y_new])
x_train = x.reset_index(drop = True)
y_train = y.reset_index(drop = True)

x_test, y_test = read_data("csv/X_mtm_test.csv","csv/Y_mtm_test.csv")

# Reduce Dimension of the data
def reduce_dim_mtm(x):
    ELGD_new = x.iloc[:,0]
    EAD_new = x.iloc[:,100:200]
    rho_new = x.iloc[:,200:300]
    c_new = x.iloc[:,300:400]
    g_new = x.iloc[:,400:500]
    D_new = x.iloc[:,500]
    return pd.concat([ELGD_new,EAD_new,rho_new,c_new,g_new,D_new],axis = 1)

x_train_small = reduce_dim_mtm(x_train)
x_test_small = reduce_dim_mtm(x_test)

In [15]:
transition_matrix = pd.read_csv("transition_matrix_SP.csv",delimiter = ",")
SP_dict = trans_matrix_to_dict(transition_matrix)

# Compute GA approx on the training set and test set

In [18]:
# x_train = x_train.iloc[1000:,:]
# x_train = x_train.reset_index(drop=True)
# x_train

In [17]:

GA_approx_List_Train = []
GA_approx_List_Test = []

# Training set

for i in range(len(x_train)):
    LGD_Vector = np.array(x_train.iloc[i,:100])
    EAD_Vector = np.array(x_train.iloc[i,100:200])
    rho_Vector = np.array(x_train.iloc[i,200:300])
    c_vector = np.array(x_train.iloc[i,300:400])
    g_Vector = np.array(x_train.iloc[i,400:500]).astype(int)
    D_vector = np.array(x_train.iloc[i,500:])
    
    GA_approx_List_Train.append(GA_GM(PD = 0, ELGD =LGD_Vector[LGD_Vector>0],
                                   A = EAD_Vector[EAD_Vector>0], M= np.array([1]*len(EAD_Vector>0)),
                      q=0.999,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g_Vector[g_Vector>0]),
                     trans_prob=SP_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = rho_Vector[EAD_Vector>0],
                     c = c_vector[EAD_Vector>0],
                    nu =0.25,               
                     S = S, # Number of States of Rating
                     default_only = False,
                      LGD_constant = False))
#Test set

for i in range(len(x_test)):
    LGD_Vector = np.array(x_test.iloc[i,:100])
    EAD_Vector = np.array(x_test.iloc[i,100:200])
    rho_Vector = np.array(x_test.iloc[i,200:300])
    c_vector = np.array(x_test.iloc[i,300:400])
    g_Vector = np.array(x_test.iloc[i,400:500]).astype(int)
    D_vector = np.array(x_test.iloc[i,500:])
    
    GA_approx_List_Test.append(GA_GM(PD = 0, ELGD =LGD_Vector[LGD_Vector>0],
                                   A = EAD_Vector[EAD_Vector>0], M= D_vector[EAD_Vector>0],
                      q=0.999,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g_Vector[g_Vector>0]),
                     trans_prob=SP_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = rho_Vector[EAD_Vector>0],
                     c = c_vector[EAD_Vector>0],
                    nu =0.25,               
                     S = S, # Number of States of Rating
                     default_only = False,
                      LGD_constant = False))

    
pd.DataFrame(GA_approx_List_Train).to_csv("csv/X_Train_act_MtM_2.csv")
pd.DataFrame(GA_approx_List_Test).to_csv("csv/X_Test_MtM_approx.csv")